In [1]:
# Import libraries
import os
import numpy as np
from beam_corot.ComplBeam import ComplBeam
from beam_corot.CoRot import CoRot
from pybevc import PyBEVC
from utils import save_load
from inertial_forces import inertial_loads_fun_v04
from input import createProjectFolder

In [3]:
# Main loop options
epsilon = 1e-3
delta_u_rel = epsilon + 1
iter_max = 20

In [4]:
# Simulation conditions
U0 = 10
TSR = 7 #Not used for now
omega = 1 # rad/s
pitch_deg = 0
pitch_rad = np.deg2rad(pitch_deg)
hub_di = 3 # See how to get it from htc file

In [2]:
# Folders and file names
inputfolder = 'examples/input_iea15mw'
projectfolder = 'examples/project_folder'
c2_file = 'c2_pos.dat'
st_file = 'IEA_15MW_RWT_Blade_st_FPM.st'
ae_file = 'IEA_15MW_RWT_ae.dat'
pc_file = 'IEA_15MW_RWT_pc.dat'

createProjectFolder(inputfolder,projectfolder,c2_file,st_file,ae_file,pc_file)



## Inertial model

In [ ]:
# Input files folder
propfile = os.path.join(os.getcwd(),'property_file.dat')

stPropertyInput = np.loadtxt(propfile, skiprows=3)

# cg_offset are a set of vectors in blade root coordinates that indicate the position of the CoG with
# respect to then nodes (centre of elasticity)
# r_node2cg = r_c22cg - r_c22node
cg_offset = 

## Structural model init

In [3]:
# Model input json file  name
f_model_json = "config.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'examples/project_folder/stru')
mainfile = os.path.join(inputfolder,f_model_json)

# Initialize beam model
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam = ComplBeam(mainfile)


-----------
Timo Mass Matrix
ComplBeam Model Created
Static analysis done


In [4]:
beam.M_mat_full

array([[ 1.22173620e+03,  6.92464424e-02,  2.99657490e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 6.92464424e-02,  1.22247162e+03, -2.53426197e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.99657490e-02, -2.53426197e-02,  1.22071170e+03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.56533717e+00,  2.68667512e-01,  7.51031361e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.68667512e-01,  3.90224401e+00,  3.37300897e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         7.51031361e-02,  3.37300897e-01,  3.51098572e-01]])

## Aerodynamic model init

In [14]:
bevc = PyBEVC()

# Setting inputs manually 
bevc.U0 = 8.0
bevc.TSR = 7


bevc.flag_a_CT = 2

inputfolder = 'examples/project_folder/aero'
bevc.from_ae_file(os.path.join(inputfolder,'ae_file.dat'))
bevc.from_pc_file(os.path.join(inputfolder,'pc_file.dat'))
bevc.from_c2_file(os.path.join(inputfolder,'c2_pos.dat'))

# Display all data as an Xarray object
xr_inp = bevc.as_xarray()
xr_inp

ValueError: x need to have the same size as s (given: np.size(s)=30, np.size(x)=34)

In [22]:
# - Init variables
pos_new = np.zeros(beam.M_mat_full.shape[0]) # The initial pos are 0. This gives no inertial forces
tip_init_pos=beam.nodeLocations[-1]
old_tip_def = np.zeros(3)
new_tip_def = np.zeros(3)
tip_def_history = []
delta_history = []
iter = 0

while abs(delta_u_rel) > epsilon and iter < iter_max:
    # Update variables
    pos_old = pos_new
    old_tip_def = new_tip_def
    tip_def_history.append(np.linalg.norm(old_tip_def))
    iter += 1

    # Calculate inertial loads
    inertial_loads = inertial_loads_fun_v03(pos_new, beam, beam.M_mat_full, hub_di, omega, pitch_rad)
    save_load(inertial_loads,projectfolder)

    # Calculate delta
    delta_u_rel = np.linalg.norm(new_tip_def - old_tip_def)/r[-1]
    delta_history.append(delta_u_rel)

UFuncTypeError: Cannot cast ufunc 'add' output from dtype('O') to dtype('float64') with casting rule 'same_kind'